В ваше распоряжение предоставлена директория users. В данной директории содержатся csv-файлы, в каждом из которых хранится информация об идентификаторах пользователей (user_id) и ссылки на их фотографии (photo_url). Файлов в директории может быть сколько угодно.

Вам необходимо написать функцию concat_user_files(path), параметром которой является path — путь до директории. Функция должна объединить информацию из предоставленных вам файлов в один DataFrame и вернуть его.

Список названий всех файлов, находящихся в директории, вы можете получить с помощью функции os.listdir(path) из модуля os. Отсортируйте полученный список, прежде чем производить объединение файлов.

Обратите внимание, что метод os.listdir() возвращает только названия файлов в указанной директории, а при чтении файла необходимо указывать полный путь до него.

Не забудьте обновить индексы результирующей таблицы после объединения.

Примечание. Учтите, что на тестовом наборе файлов в результате объединения могут возникнуть дубликаты, от которых необходимо будет избавиться.



In [1]:
import pandas as pd
import os

def concat_user_files(path):
    data = pd.DataFrame()
    file_names = os.listdir(path)
    file_names.sort()
    for file in file_names:
        tmp_data = pd.read_csv(path + '/' + file)
        data = pd.concat([data, tmp_data], axis=0, ignore_index=True)
    data = data.drop_duplicates()
    return data


Результат нашего объединения таблиц с рейтингами вы можете найти здесь (csv-файл находится в zip-архиве — распакуйте архив, прежде чем продолжать работу!)

In [20]:
import pandas as pd
joined = pd.read_csv('data/ratings_movies.csv')
display(joined.tail())

,Unnamed: 0,userId,movieId,rating,date,title,genres
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi
100835,100835,610,170875,3.0,2017-05-03 21:20:15,The Fate of the Furious (2017),Action|Crime|Drama|Thriller


Для решения задач нам понадобится выделить из признака title год выпуска фильма. Для этого напишем функцию get_year_release(arg).

In [21]:
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.

У скольких фильмов не указан год их выпуска?

In [22]:
joined['year_release'] = joined['title'].apply(get_year_release)
joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?

В качестве ответа запишите название этого фильма без указания года его выпуска.

In [26]:
mask = joined['year_release'] == 1999
joined[mask].groupby('title')['rating'].mean().sort_values()

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?

Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).

In [33]:
mask = joined['year_release'] == 2010
joined[mask].groupby('genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов? В качестве ответа запишите идентификатор этого пользователя.
     Подсказка (1 из 3): Сгруппируйте таблицу по пользователям и найдите число уникальных жанров для каждого пользователя.
    Подсказка (2 из 3): Отсортируйте результат по убыванию.
    Подсказка (3 из 3): Индекс первой строки результирующей таблицы будет являться ответом.

In [65]:
joined.groupby('userId')['genres'].nunique().sort_values(ascending=False)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.

В качестве ответа укажите идентификатор этого пользователя.

Чтобы рассчитать несколько параметров для каждого пользователя (количество оценок и среднюю оценку), можно воспользоваться методом agg() на сгруппированных данных.
    Подсказка (1 из 4): Сгруппируйте таблицу по пользователям (userId).
    Подсказка (2 из 4): Для каждого пользователя рассчитайте количество выставленных им оценок и средний рейтинг.
    Подсказка (3 из 4): Отсортируйте таблицу по возрастанию количества оценок и по убыванию среднего рейтинга.
    Подсказка (4 из 4): Индекс первой строки результирующей таблицы будет являться ответом.

In [66]:
joined.groupby('userId')['rating'].agg(
    ['count', 'mean']
).sort_values(['count', 'mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг (среднее по столбцу rating), и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.

Запишите сочетание так же, как оно указано в таблице (через разделитель |, без пробелов).
    Подсказка (1 из 6): Произведите фильтрацию по условию «год выпуска 2018».
    Подсказка (2 из 6): Сгруппируйте отфильтрованную таблицу по жанрам и найдите средний рейтинг, а также количество выставленных оценок с помощью метода agg().
    Подсказка (3 из 6): Сохраните сгруппированную таблицу.
    Подсказка (4 из 6): Произведите фильтрацию полученной таблицы по условию «количество оценок > 10».
    Подсказка (5 из 6): Отсортируйте результат по столбцу со средним значением по убыванию.
    Подсказка (6 из 6): Индекс первой строки результирующей таблицы будет являться ответом.

In [69]:
mask = joined['year_release'] == 2018
grouped = joined[mask].groupby('genres')['rating'].agg(
    ['mean', 'count']
)
grouped[grouped['count']>10].sort_values(
    by='mean',
    ascending=False
)

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


Добавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра. Выберите верные варианты ответа, исходя из построенной таблицы:
    Подсказка (1 из 3): Переведите признак date в формат datetime.
    Подсказка (2 из 3): Выделите признак года выставления оценки с помощью атрибута аксессора dt year, результат занесите в столбец year_rating.
    Подсказка (3 из 3): Составьте сводную таблицу, индексами которой являются годы выхода фильмов (year_rating), столбцами — жанры (genres), а в качестве значений выступают средние оценки фильмов.

In [85]:

joined['date'] = pd.to_datetime(joined['date'])
joined['year_rating'] = joined['date'].dt.year
pivot = joined.pivot_table(
    index='year_rating',
    columns='genres',
    values='rating',
    aggfunc='mean'
)
display(pivot.loc[:, ['Comedy', 'Animation|Children|Mystery' , 'Action|Adventure|Animation|Children|Comedy|IMAX', 'Action|Adventure']])

genres,Comedy,Animation|Children|Mystery,Action|Adventure|Animation|Children|Comedy|IMAX,Action|Adventure
year_rating,,,,
1996,3.228571,NaN,NaN,3.454545
1997,3.409091,NaN,NaN,4.150000
1998,3.000000,NaN,NaN,4.200000
1999,3.606061,NaN,NaN,4.000000
2000,3.141291,NaN,NaN,3.738462
2001,3.318408,NaN,NaN,3.500000
2002,3.198556,NaN,NaN,4.304348
2003,3.120066,NaN,NaN,3.277778
2004,3.356877,NaN,NaN,4.136364


Дано два файла:

        orders.csv, содержащий данные о заказах;
        products.csv, содержащий данные о товарах.
Прочитайте данные файлы, посмотрите на содержимое таблиц и проанализируйте его.

Подумайте, как связаны данные таблицы (какими ключевыми столбцами).

In [2]:
import pandas as pd
order_df = pd.read_csv('data/orders.csv', ';')
product_df = pd.read_csv('data/products.csv', ';')
display(order_df)
display(product_df)

C:\temp\ipykernel_2660\1917792197.py:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  order_df = pd.read_csv('data/orders.csv', ';')
C:\temp\ipykernel_2660\1917792197.py:3: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  product_df = pd.read_csv('data/products.csv', ';')


,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3


,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR
5,86,"Носки Простые, муж",45,RUR
6,91,"Носки Честные, муж",50,RUR
7,103,"Носки Подарочные, муж",199,RUR
8,104,"Носки Подарочные, жен",249,RUR
9,124,Носки беговые Camino,999,RUR


In [3]:
orders_products = order_df.merge(
    product_df, 
    left_on='ID товара',
    right_on='Product_ID',
    how='left')

display(orders_products)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86.0,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104.0,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104.0,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104.0,"Носки Подарочные, жен",249.0,RUR
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124.0,Носки беговые Camino,999.0,RUR
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,91.0,"Носки Честные, муж",50.0,RUR
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,103.0,"Носки Подарочные, муж",199.0,RUR
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,104.0,"Носки Подарочные, жен",249.0,RUR


Какой идентификатор (Order ID) имеет заказ, для которого не оказалось информации о товаре?

In [13]:
#orders_products[orders_products["Name"].isnull()]["Order ID"]
orders_products[orders_products['Name'].isna()]

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
17,01.01.2001 00:00:00,0,1,"Оплачен, формируется к отправке",Да,Нет,Нет,666,1,NaN,NaN,NaN,NaN


На какой товар была произведена отмена?

В качестве ответа запишите название этого товара (Name).

In [14]:
orders_products[orders_products['Отменен'] == 'Да']['Name']

6    Носки беговые Camino
Name: Name, dtype: object

Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
В ответ запишите идентификатор этого покупателя (ID Покупателя).
Прибыль состоит только из оплаченных заказов и рассчитывается как количество купленного товара, умноженное на его цену.
 
    Подсказка (1 из 2): Cоздадим признак прибыли за один заказ - profit - количество товара * цену товара. Затем произведем фильтрацию по условию оплаченного заказа.

In [15]:
orders_products['Profit'] = orders_products['Price'] * orders_products['Количество'] 
orders_products[orders_products['Оплачен'] == 'Да'].groupby('ID Покупателя')['Profit'].sum().sort_values(ascending=False)


ID Покупателя
7    17096.0
5    13043.0
8     1344.0
1        0.0
Name: Profit, dtype: float64